In [925]:
import pandas as pd
import numpy as np
from collections import Counter

In [926]:
prediction_files = [
        "output_1.csv",
        "output_2.csv",
        "output_3.csv",
        "output_4.csv",
        "output_5.csv",
        "output_9.csv",
        "output_8.csv",
        "temp4.csv",
        "temp6.csv"
        ]

In [927]:
predictions = []
for file in prediction_files:
    df = pd.read_csv(file)
    predictions.append(df)
    print(f"Loaded: {file}, Shape: {df.shape}")

Loaded: output_1.csv, Shape: (869, 8)
Loaded: output_2.csv, Shape: (869, 8)
Loaded: output_3.csv, Shape: (869, 8)
Loaded: output_4.csv, Shape: (869, 8)
Loaded: output_5.csv, Shape: (869, 8)
Loaded: output_9.csv, Shape: (869, 8)
Loaded: output_8.csv, Shape: (869, 8)
Loaded: temp4.csv, Shape: (869, 8)
Loaded: temp6.csv, Shape: (869, 8)


In [928]:
len(predictions)

9

# soft voting (확률값 평균)

In [929]:
first_df = predictions[0]
result_df = pd.DataFrame()
result_df['id'] = first_df['id']
final_answers_v1 = [] #원래 soft voting방법 (확률의 평균을 계산)
final_probs_v1 = []
final_answers_v2 = [] #확률의 평균을 계산한 후에 결정되는 답의 확률값이 threshold 이하일 경우 최대 확률을 가지는 답안으로 변경
final_probs_v2 = []
threshold=0.5

for idx, row_id in enumerate(result_df['id']):
    current_probs = []
    origin_answer = []
    for pred_df in predictions:
        pred_row = pred_df[pred_df['id'] == row_id].iloc[0]
        probs = [pred_row[col] for col in pred_row.index if col.startswith('prob_')]
        probs = [0 if pd.isna(p) else p for p in probs]
        current_probs.append(probs)
        
        max_prob = max(probs)
        answer = str(np.argmax(probs) + 1)
        origin_answer.append((answer, max_prob)) #(답변, 최대확룰)
        
    #각 선택지의 확률 평균 계산
    avg_probs = np.mean(current_probs, axis=0)
    
    answer_v1 = str(np.argmax(avg_probs) + 1)
    final_answers_v1.append(answer_v1)
    final_probs_v1.append(avg_probs)
        
    ensemble_max_prob = max(avg_probs)
    if ensemble_max_prob < threshold:
        highest_prob = 0
        for answer_tuple in origin_answer:
            if answer_tuple[1] > highest_prob:
                highest_prob = answer_tuple[1]
                best_answer = answer_tuple
        
        final_answer = best_answer[0]  # 가장 확신도 높은 모델의 답
        idx = origin_answer.index(best_answer)
        final_prob = current_probs[idx]
    else:
        final_answer = str(np.argmax(avg_probs) + 1)
        final_prob = avg_probs
        
    final_answers_v2.append(final_answer)
    final_probs_v2.append(final_prob)

In [930]:
result_v1 = result_df.copy()
result_v1['answer'] = final_answers_v1
for i in range(len(final_probs_v1[0])):
    result_v1[f'prob_{i+1}'] = [probs[i] for probs in final_probs_v1]

In [931]:
result_v2 = result_df.copy()
result_v2['answer'] = final_answers_v2
for i in range(len(final_probs_v2[0])):
    result_v2[f'prob_{i+1}'] = [probs[i] for probs in final_probs_v2]

In [932]:
result_v1.to_csv("ensemble_output_soft.csv")
result_v2.to_csv("ensemble_output_soft_v2.csv")

# Hard Voting (다수결)
- 동점이 있는 경우 동점이 있는 선택지 중 가장 확률값이 높은 것 선택

In [933]:
# first_df = predictions[0]
# result_df = pd.DataFrame()
# result_df['id'] = first_df['id']

# final_answers_hv_v1 = []
# final_probs = []
# for idx in range(len(first_df)):
#     votes = [pred_df.iloc[idx]['answer'] for pred_df in predictions]
#     vote_counts = Counter(votes)
#     most_commons = vote_counts.most_common()

#     #최빈값 계산
#     max_count = most_commons[0][1]
#     top_predictions = [pred for pred, count in most_commons if count == max_count]

#     #답이 하나로 정해진 경우
#     if len(top_predictions) == 1:
#         final_answers_hv_v1.append(top_predictions[0])
#         probs = []
#         for i in range(1, len(first_df.filter(like='prob_').columns) + 1):
#             prob_values = [float(pred_df.iloc[idx][f'prob_{i}']) for pred_df in predictions]
#             probs.append(np.mean(prob_values))
#         final_probs.append(probs)
#     #공동1등이 나오는 경우
#     else:
#         probs = {}
#         for ans in top_predictions:
#             #각 모델 별 확률값 중 최대값 선택
#             probs[ans] = max(float(pred_df.iloc[idx][f'prob_{ans}']) for pred_df in predictions)

#         # 가장 높은 확률값을 가지는 답안 선택
#         selected_answer = max(probs.items(), key=lambda x: x[1])[0]
#         #prob = probs[selected_answer]
#         final_answers_hv_v1.append(selected_answer)
#         prob_list = []
#         for i in range(1, len(first_df.filter(like='prob_').columns) + 1):
#             prob_values = [float(pred_df.iloc[idx][f'prob_{i}']) for pred_df in predictions]
#             prob_list.append(np.mean(prob_values))
#         final_probs.append(prob_list)

# result_df['answer'] = final_answers_hv_v1
# final_probs = np.array(final_probs)
# for i in range(final_probs.shape[1]):
#     result_df[f'prob{i+1}'] = final_probs[:, i]

In [934]:
# vote_counts.keys()

In [935]:
# result_df.to_csv("ensemble_output_hard1.csv")

# Hard Voting (확률값 더하기)

In [936]:
# first_df = predictions[0]
# result_df = pd.DataFrame()
# result_df['id'] = first_df['id']

# final_answers_hv_v2 = []
# final_probs = []
# for idx in range(len(first_df)):
#     votes = [pred_df.iloc[idx]['answer'] for pred_df in predictions]
#     vote_counts = Counter(votes)
#     most_commons = vote_counts.most_common()
    
#     #최빈값 계산
#     max_count = most_commons[0][1]
#     top_predictions = [pred for pred, count in most_commons if count == max_count]
    
#     #답이 하나로 정해진 경우
#     if len(top_predictions) == 1:
#         final_answers_hv_v2.append(top_predictions[0])
#         probs = []
#         for i in range(1, len(first_df.filter(like='prob_').columns) + 1):
#             prob_values = [float(pred_df.iloc[idx][f'prob_{i}']) for pred_df in predictions]
#             probs.append(np.mean(prob_values))
#         final_probs.append(probs)
#     #공동1등이 나오는 경우
#     else:
#         probs = {}
#         for ans in top_predictions:
#             #각 모델의 확률값을 모두 더함
#             probs[ans] = sum(float(pred_df.iloc[idx][f'prob_{ans}']) for pred_df in predictions)
        
#         # 확률 합이 가장 높은 답안 선택
#         selected_answer = max(probs.items(), key=lambda x: x[1])[0]
#         final_answers_hv_v2.append(selected_answer)
        
#         prob_list = []
#         for i in range(1, len(first_df.filter(like='prob_').columns) + 1):
#             prob_values = [float(pred_df.iloc[idx][f'prob_{i}']) for pred_df in predictions]
#             prob_list.append(np.mean(prob_values))
#         final_probs.append(prob_list)
    
# result_df['answer'] = final_answers_hv_v2
# final_probs = np.array(final_probs)
# for i in range(final_probs.shape[1]):
#     result_df[f'prob_{i+1}'] = final_probs[:, i]

In [937]:
# result_df.to_csv("ensemble_output_hard2.csv")